# Задача на программирование

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью ф-и numpy.inv.

# Предварительные сведения

Векторные нормы:

$||u||_{\inf} = \max_i|u_i|$

$||u||_1 = \sum_i |u_i|$

$||u||_2 = \left(\sum_i |u_i|^2 \right)^{\frac{1}{2}}$

Матричные нормы:

$||A||_{\inf} = \max_i \sum_j |a_{ij}|$

$||A||_1 = \max_j \sum_i |a_{ij}|$

$||A||_2 = \left(\max_i \lambda_i(A A^*) \right)^{\frac{1}{2}}$

Контрольный вопрос: какова будет вторая норма матрицы, если матрица самосопряженная?

In [1]:
import numpy as np
import numpy.linalg as la

A = np.array([[1,2],[3,4]])
v = range(0,3)
Vander = np.vander(v)
print('norm_1 = ', la.norm(Vander, 1))
print('norm_2 = ', la.norm(Vander, 2))
print('norm_inf = ', la.norm(Vander, np.inf))
Vander

norm_1 =  5.0
norm_2 =  4.8449585245
norm_inf =  7.0


array([[0, 0, 1],
       [1, 1, 1],
       [4, 2, 1]])

Обусловленность:
$$(A+\delta A)u = f + \delta f$$
$$\frac{||\delta u||}{||u||}\le \frac{\mu}{1-\mu\frac{||\delta A||}{||A||}} \left(\frac{||\delta f||}{||f||}+\frac{||\delta A||}{||A||}\right)$$

$\mu$ - число обусловленности матрицы A, $\mu(A) = ||A^{-1}||\cdot||A||$, $\mu \ge 1$.



# Пример проблемы использования метода Гаусса

In [2]:
import numpy as np
import numpy.linalg as la

def gauss( A, b ):
    n = b.size
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i][k]!=0:
                c = A[i][k]/A[k][k]
                A[i][k+1:n] = A[i][k+1:n] - c*A[k][k+1:n]
                b[i] = b[i] - c*b[k]
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k][k+1:n],b[k+1:n]))/A[k][k];
    return b

#все числа в представлены как вещественные
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]]);
b1 = np.array([0., 0., 1.]);

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

print('u1 = ', gauss(A1, b1))#la.solve(A1, b1))
print('u2 = ', gauss(A2, b2))#la.solve(A2, b2))

mu1 =  16.3937316223
mu2 =  16.3937316223
u1 =  [ 0.55511151  0.25        0.25      ]
u2 =  [ 1.  1.  1.]


# Пример задачи на использование метода простой итерации (МПИ)

Для СЛАУ $Ax = b$, где $b = (6, -6, 1)^T$ и
$$
A = \begin{pmatrix} 
6 & 0  & 7 \\
0 & 6 & 6\\
7 & 6 & 18 \\
\end{pmatrix}
$$

1. вычислить число обусловленности в трех нормах.
2. для заданой относительной погрешности правой части $\frac{||\delta b||_2}{||b||_2} = 0.01$ найти границы для относительной погрешности $\frac{||\delta x||_2}{||x||_2}$ решения системы.
3. исследовать на сходимость и оценить скорость сходимости МПИ $x_{k+1} = (E - \tau A)x_k + \tau b$ при $\tau = 0.01$.
4. найти оптимальный параметр МПИ $\tau_{опт}$ и дать оценку сходимости при этом $\tau_{опт}$.

Замечание: заметим, что в данном случае матрица А самосопряженная, поэтому удобно считать вторую норму матрицы. В противном случае можно было бы домножить систему слева на $A^T$.

Решение.
1. Вычисляется по формулам выше. Во второй норме число обусловленности $\mu_3 = \dfrac{\lambda_{max}(A)}{\lambda_{min}(A)} = 23$, так как $\lambda_{max}(A^{-1}) = \dfrac{1}{\lambda_{min}(A)}$.
2. По формуле, связывающей относительную ошибку, ошибку входных данных и число обусловленности можно получить $\frac{||\delta x||_2}{||x||_2} \le \mu_2 \frac{||\delta b||_2}{||b||_2} = 23\cdot 0.01 = 0.23$
3. Для симметричной матрицы скорость сходимости можно определить как $q = ||B||_2 = ||E - \tau A||_2 = \max_i |1 - \tau \lambda_i|$. Условие сходимости q < 1 сводится к решению системы:
$$
\begin{array}{rcl}
{|1 - \tau \lambda_{max}|<1} \\
{|1 - \tau \lambda_{min}|<1} 
\end{array}
$$ 
решение которой дает условие сходимости $0<\tau<\dfrac{2}{\lambda_{max}}$. Скорость сходимости при $\tau = 0.01$:
$$
q = \max\left( |1 - 0.01\cdot 1|, |1 - 0.01\cdot 6|, |1 - 0.01\cdot 23|\right) = 0.99
$$
4. Для симметричной положительно определенной матрицы оптимальный итерационный параметр 
$$
\tau_{опт} = \frac{2}{\lambda_{max} + \lambda_{min}} = \frac{1}{12},
$$
скорость сходимости 
$$
q = \frac{\lambda_{max} - \lambda_{min}}{\lambda_{max} + \lambda_{min}} = \frac{11}{12}
$$

Замечание: оптимальный параметр находится из условия $\min_{\tau}\max_i |1 - \tau \lambda_i|$ и не учитывает правую часть решаемой системы. Поэтому скорость сходимости при выборе оптимального параметра будет достаточно хорошей, но не наилучшей.

Замечание: в методе Гаусса без выбора главного члена при больших размерах матрицы $n$ требуемое число операций для решения системы $\approx \frac{2}{3}n^3$, в МПИ $\approx 2n^2\cdot I$, где $I$ - число итераций. То есть при $I<\frac{n}{3}$ МПИ лучше. Обычно $I<<n$.

Замечание: погрешность, вносимая в решение из-за конечной разрядности мантиссы не зависит от кол-ва итераций $\epsilon \le \frac{\delta}{1-q}$.

# Оценка относительной ошибки решения системы при известной правой части

Для СЛАУ $Ax = f$, где $f = (1, f_2)^T$ и
$$
A = \begin{pmatrix} 
2 & 1  \\
1 & 2 \\
\end{pmatrix}
$$
матрица А задана точно, а правая часть f может иметь погрешность $\delta f$. Найти такое $f_2$, при котором выполняется неравенство $\frac{||\delta x||}{||x||} = \frac{||\delta f||}{||f||}$. 

Решение. 
Рассмотрим соотношение $\frac{||\delta x||}{||x||} \le \nu(f) \frac{||\delta f||}{||f||}$. Найдем верхнюю грань $\nu(f)$ по всем возмущения правой части. 
$$
\nu(f) = \sup_{\delta f} \frac{||\delta x||}{||x||}\frac{||f||}{||\delta f||} = \frac{||f||}{||u||}||A^{-1}||
$$
Приравниваем к единицы и получаем соотношение на $f_2$ для выполнения требуемой оценки.

In [5]:
import scipy
import scipy.linalg  

import numpy as np
import numpy.linalg as la

def lu_decomposition(A):
    n = len(A)

    L = np.zeros((n,n))
    U = np.zeros((n,n))
                                                                                                                                                                                                                   
    for j in range(n):                                                                                                                                                                                                  
        L[j][j] = 1.0
                                                                                                                                                                                    
        for i in range(j+1):
            s1 = 0
            for k in range(i):
                s1 += U[k][j] * L[i][k]
            #s1 = sum(U[k][j] * L[i][k] for k in range(i))
            U[i][j] = A[i][j] - s1
                                                                                                                                                                  
        for i in range(j, n):
            s2 = 0
            for k in range (j):
                s2 += U[k][j] * L[i][k]
            #s2 = sum(U[k][j] * L[i][k] for k in range(j))
            L[i][j] = (A[i][j] - s2) / U[j][j]
            
    return (L, U)


A = scipy.array([ [1, 1, 1], [0, 1, 2], [7, 1, 4] ])
P0, L0, U0 = scipy.linalg.lu(A)
L, U = lu_decomposition(A)

print("A:")
print(A)
'''
print("\nL:")
print(L)
print("L0:")
print(L0)

print("\nU:")
print(U)
print("U0:")
print(U0)
'''

N = len(A)
X = np.zeros((N,N))
for i in range(0, N):
    e = np.zeros(N)
    e[i] = 1
    
    y = la.solve(L, e)
    x = la.solve(U, y)
    
    for j in range(0, N):
        X[j][i] = x[j]
print("\nX:")
print(X)

print("\nTrue invert matrix\n", la.inv(A))

A:
[[1 1 1]
 [0 1 2]
 [7 1 4]]

X:
[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]

True invert matrix
 [[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]]
